In [60]:
# Importando bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.optimize import linprog
import numpy as np

In [62]:
ev_population_data = pd.read_csv('Electric_Vehicle_Population_Data.csv')
charging_stations_data = pd.read_csv('Electric_Vehicle_Charging_Stations.csv')

In [64]:
# Tratando valores ausentes no dataset de veículos elétricos
ev_population_data[['County', 'City', 'Postal Code', 'Electric Utility']] = ev_population_data[
    ['County', 'City', 'Postal Code', 'Electric Utility']
].fillna('Unknown')

ev_population_data = ev_population_data.dropna(subset=['Vehicle Location'])

In [65]:
# Extraindo coordenadas geográficas dos veículos
ev_population_data[['Latitude', 'Longitude']] = ev_population_data['Vehicle Location'].str.extract(
    r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)'
).astype(float)

In [66]:
# Extraindo coordenadas geográficas das estações de carregamento
charging_stations_data[['Latitude', 'Longitude']] = charging_stations_data['New Georeferenced Column'].str.extract(
    r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)'
).astype(float)

Previsão de demanda

In [67]:
# Criar um conjunto de dados fictício de exemplo (
# Criando uma coluna 'Time' (horário) e 'Demand' (demanda energética) no dataset de estações
charging_stations_data['Time'] = pd.date_range(start='2024-01-01', periods=len(charging_stations_data), freq='H')
charging_stations_data['Demand'] = np.random.randint(10, 50, size=len(charging_stations_data))  # Geração aleatória

<ipython-input-67-3c22a7d70f3d>:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  charging_stations_data['Time'] = pd.date_range(start='2024-01-01', periods=len(charging_stations_data), freq='H')


In [68]:
# Adicionar variáveis temporais
charging_stations_data['Hour'] = pd.to_datetime(charging_stations_data['Time']).dt.hour
charging_stations_data['DayOfWeek'] = pd.to_datetime(charging_stations_data['Time']).dt.dayofweek

In [69]:
# Variáveis independentes (X) e dependente (y)
X = charging_stations_data[['Latitude', 'Longitude', 'Hour', 'DayOfWeek']]
y = charging_stations_data['Demand']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
# Treinar modelo de regressão Random Forest
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Após treinar o modelo, criamos uma equação para otimizar a carga por veículos

In [88]:
# Definindo as variáveis de entrada
custo_por_kw = np.random.uniform(0.1, 0.5, size=len(X_test))  # Custo fictício por kW
energia_necessaria = np.random.randint(5, 20, size=len(X_test))  # Supondo a energia necessária por veículo
energia_max = energia_necessaria * 2  # Suposto limite máximo de energia por veículo

In [84]:
# A função tem o objetivo de minimizar o custo total de energia, encontrando o custo por kw de energia para um veículo
objective = custo_por_kw

In [89]:
# Definindo os limites das variáveis:
constraints = [[1] * len(energia_max)]  # Somatório de energia tem que ser menor ou igual a energia máxima
bounds = [(0, energia_max[i]) for i in range(len(energia_max))] #O valor da energia alocada deve estar entre 0 e o limite máximo de energia máxima

In [91]:
# Resolvendo a equação usando o objetivo que é minimizar o custo total de energia, e os limites definidos
result = linprog(c=objective, A_ub=[-np.ones(len(energia_max))], b_ub=[-sum(energia_max)], bounds=bounds)

In [93]:
# Resultados da otimização
if result.success:
    print(f"Solução Otimizada Encontrada: {result.x}")
    print(f"Custo Total Minimizado: {result.fun:.2f}")
else:
    print("Otimização não foi bem-sucedida.")

Solução Otimizada Encontrada: [38. 22. 10. 16. 22. 12. 16. 10. 22. 28. 36. 14. 24. 26. 36. 26. 28. 36.
 32. 12. 26. 28. 14. 26. 14. 38. 36. 38. 16. 24. 32. 28. 16. 18. 26. 18.
 38. 30. 34. 24. 28. 38. 22. 20. 26. 14. 16. 18. 38. 26. 14. 16. 38. 14.
 30. 32. 16. 12. 22. 18. 32. 24. 26. 18. 36. 38. 38. 16. 38. 28. 38. 34.
 34. 36. 36. 26. 36.]
Custo Total Minimizado: 581.73
